In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/desktop/test_write.txt',delimiter = ',',dtype = None)


xy= xy[:,:27]

seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 1500

train_size = int(len(xy)*0.7)
train_set = xy[:7]
test_set = xy[7:]


def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)
trainX, trainY = build_dataset(train_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

X = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y = tf.placeholder(tf.float32,[None,25])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
outputs,_states = tf.nn.dynamic_rnn(cell,X,dtype = tf.float32)
Y_pred = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim,activation_fn = None)

loss =tf.reduce_mean(tf.square(Y_pred-Y))
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.5
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss = sess.run([train,loss],feed_dict={X:trainX, Y:trainY})
        print("[step: {}] loss: {}".format(i,step_loss))
        
    test_predict = sess.run(Y_pred, feed_dict = {X:testX})
    rmse_val = sess.run(rmse, feed_dict={targets: testY,predictions: test_predict})
    print("RMSE: {}".format(rmse_val))
    print("pred: {}".format(test_predict[-1,:]))
    print("real: {}".format(testY[-1,:]))
    
    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.3)
    plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.3)
    plt.legend()
    plt.show()


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 11.284062385559082
[step: 1] loss: 9.70776653289795
[step: 2] loss: 8.50062084197998
[step: 3] loss: 7.465725898742676
[step: 4] loss: 6.520405292510986
[step: 5] loss: 5.659430980682373
[step: 6] loss: 4.896528720855713
[step: 7] loss: 4.222663879394531
[step: 8] loss: 3.627551555633545
[step: 9] loss: 3.1056289672851562
[step: 10] loss: 2.652656316757202
[step: 11] loss: 2.262049674987793
[step: 12] loss: 1.9242448806762695
[step: 13] loss: 1.6318542957305908
[step: 14] loss: 1.3789689540863037
[step: 15] loss: 1.1606454849243164
[step: 16] loss: 0.9726731777191162
[step: 17] loss: 0.8113548755645752
[step: 18] loss: 0.6734438538551331
[step: 19] loss: 0.5561686158180237
[step: 20] loss: 0.45723360776901245
[step: 21] loss: 0.37472751736640

[step: 252] loss: 1.534939363112553e-11
[step: 253] loss: 1.444841555425791e-11
[step: 254] loss: 1.2601190924055317e-11
[step: 255] loss: 1.2819123366980545e-11
[step: 256] loss: 1.2563623752459563e-11
[step: 257] loss: 1.2980818676899819e-11
[step: 258] loss: 1.1865433115210156e-11
[step: 259] loss: 1.1798287173625521e-11
[step: 260] loss: 1.1488206219656405e-11
[step: 261] loss: 9.839919677678832e-12
[step: 262] loss: 9.740248538281371e-12
[step: 263] loss: 8.64693386337656e-12
[step: 264] loss: 7.613714346488276e-12
[step: 265] loss: 6.490133517417229e-12
[step: 266] loss: 5.458887248482869e-12
[step: 267] loss: 4.7227651477077526e-12
[step: 268] loss: 3.941016783748363e-12
[step: 269] loss: 3.135271989945787e-12
[step: 270] loss: 2.773801154043465e-12
[step: 271] loss: 2.1402751532867414e-12
[step: 272] loss: 1.7241103293305637e-12
[step: 273] loss: 1.4914785986114287e-12
[step: 274] loss: 1.3725576664819217e-12
[step: 275] loss: 1.3035178405423897e-12
[step: 276] loss: 1.29350751

[step: 470] loss: 4.9960038093365514e-18
[step: 471] loss: 4.9960038093365514e-18
[step: 472] loss: 4.9960038093365514e-18
[step: 473] loss: 4.9960038093365514e-18
[step: 474] loss: 4.9960038093365514e-18
[step: 475] loss: 4.9960038093365514e-18
[step: 476] loss: 4.9960038093365514e-18
[step: 477] loss: 4.9960038093365514e-18
[step: 478] loss: 4.9960038093365514e-18
[step: 479] loss: 4.9960038093365514e-18
[step: 480] loss: 4.9960038093365514e-18
[step: 481] loss: 4.9960038093365514e-18
[step: 482] loss: 4.9960038093365514e-18
[step: 483] loss: 4.9960038093365514e-18
[step: 484] loss: 4.9960038093365514e-18
[step: 485] loss: 4.9960038093365514e-18
[step: 486] loss: 4.9960038093365514e-18
[step: 487] loss: 4.9960038093365514e-18
[step: 488] loss: 4.9960038093365514e-18
[step: 489] loss: 4.9960038093365514e-18
[step: 490] loss: 4.9960038093365514e-18
[step: 491] loss: 4.9960038093365514e-18
[step: 492] loss: 4.9960038093365514e-18
[step: 493] loss: 4.9960038093365514e-18
[step: 494] loss

[step: 718] loss: 1.9984015237346206e-17
[step: 719] loss: 1.9984015237346206e-17
[step: 720] loss: 1.9984015237346206e-17
[step: 721] loss: 6.716849510740434e-17
[step: 722] loss: 6.716849510740434e-17
[step: 723] loss: 6.716849510740434e-17
[step: 724] loss: 6.716849510740434e-17
[step: 725] loss: 6.716849510740434e-17
[step: 726] loss: 1.3877787807814457e-17
[step: 727] loss: 1.3877787807814457e-17
[step: 728] loss: 1.3877787807814457e-17
[step: 729] loss: 1.3877787807814457e-17
[step: 730] loss: 1.3877787807814457e-17
[step: 731] loss: 1.3877787807814457e-17
[step: 732] loss: 1.3877787807814457e-17
[step: 733] loss: 1.3877787807814457e-17
[step: 734] loss: 1.3877787807814457e-17
[step: 735] loss: 1.3877787807814457e-17
[step: 736] loss: 1.3877787807814457e-17
[step: 737] loss: 1.3877787807814457e-17
[step: 738] loss: 7.993606094938482e-17
[step: 739] loss: 7.993606094938482e-17
[step: 740] loss: 7.993606094938482e-17
[step: 741] loss: 7.993606094938482e-17
[step: 742] loss: 7.99360

[step: 952] loss: 3.552713599391162e-17
[step: 953] loss: 3.552713599391162e-17
[step: 954] loss: 3.552713599391162e-17
[step: 955] loss: 3.552713599391162e-17
[step: 956] loss: 3.552713599391162e-17
[step: 957] loss: 4.496403262966774e-17
[step: 958] loss: 4.496403262966774e-17
[step: 959] loss: 4.496403262966774e-17
[step: 960] loss: 4.496403262966774e-17
[step: 961] loss: 4.496403262966774e-17
[step: 962] loss: 4.496403262966774e-17
[step: 963] loss: 3.552713599391162e-17
[step: 964] loss: 3.552713599391162e-17
[step: 965] loss: 3.552713599391162e-17
[step: 966] loss: 3.552713599391162e-17
[step: 967] loss: 3.552713599391162e-17
[step: 968] loss: 3.552713599391162e-17
[step: 969] loss: 3.552713599391162e-17
[step: 970] loss: 3.552713599391162e-17
[step: 971] loss: 3.552713599391162e-17
[step: 972] loss: 3.552713599391162e-17
[step: 973] loss: 3.552713599391162e-17
[step: 974] loss: 3.552713599391162e-17
[step: 975] loss: 4.496403262966774e-17
[step: 976] loss: 4.496403262966774e-17


[step: 1189] loss: 6.716849510740434e-17
[step: 1190] loss: 6.716849510740434e-17
[step: 1191] loss: 6.716849510740434e-17
[step: 1192] loss: 1.9984015237346206e-17
[step: 1193] loss: 1.9984015237346206e-17
[step: 1194] loss: 1.9984015237346206e-17
[step: 1195] loss: 1.9984015237346206e-17
[step: 1196] loss: 1.9984015237346206e-17
[step: 1197] loss: 1.9984015237346206e-17
[step: 1198] loss: 1.9984015237346206e-17
[step: 1199] loss: 6.716849510740434e-17
[step: 1200] loss: 6.716849510740434e-17
[step: 1201] loss: 6.716849510740434e-17
[step: 1202] loss: 6.716849510740434e-17
[step: 1203] loss: 1.9984015237346206e-17
[step: 1204] loss: 1.9984015237346206e-17
[step: 1205] loss: 1.9984015237346206e-17
[step: 1206] loss: 1.9984015237346206e-17
[step: 1207] loss: 1.9984015237346206e-17
[step: 1208] loss: 1.9984015237346206e-17
[step: 1209] loss: 1.9984015237346206e-17
[step: 1210] loss: 6.716849510740434e-17
[step: 1211] loss: 6.716849510740434e-17
[step: 1212] loss: 6.716849510740434e-17
[s

[step: 1416] loss: 6.716849510740434e-17
[step: 1417] loss: 6.716849510740434e-17
[step: 1418] loss: 1.9984015237346206e-17
[step: 1419] loss: 1.9984015237346206e-17
[step: 1420] loss: 1.9984015237346206e-17
[step: 1421] loss: 1.9984015237346206e-17
[step: 1422] loss: 1.9984015237346206e-17
[step: 1423] loss: 1.9984015237346206e-17
[step: 1424] loss: 6.716849510740434e-17
[step: 1425] loss: 6.716849510740434e-17
[step: 1426] loss: 6.716849510740434e-17
[step: 1427] loss: 1.9984015237346206e-17
[step: 1428] loss: 1.9984015237346206e-17
[step: 1429] loss: 1.9984015237346206e-17
[step: 1430] loss: 1.9984015237346206e-17
[step: 1431] loss: 1.9984015237346206e-17
[step: 1432] loss: 1.9984015237346206e-17
[step: 1433] loss: 6.716849510740434e-17
[step: 1434] loss: 6.716849510740434e-17
[step: 1435] loss: 6.716849510740434e-17
[step: 1436] loss: 1.3877787807814457e-17
[step: 1437] loss: 1.3877787807814457e-17
[step: 1438] loss: 1.3877787807814457e-17
[step: 1439] loss: 1.3877787807814457e-17
